## Experimental Work and The Transferability Principle
```In this exercise you will experience with real experimental work and will meet a very interesting issue - adversarial AI and the transferability principle. You will define experiments and measures for success, and will execute those experiments. It is of great importance that you will discuss this exercise with your tutor, even during the work on the exercise.```

```~Ohad Amosi & Ittai Haran```

```Read the paper "Adversarial Examples Are Not Bugs, They Are Features", which you can find in this exercise directory. Read it thoroughly. Make sure you understand how the datasets of experiments #1 and #2 were generated. As you might tell, the paper reports very interesting results.```

```If you are not familiar with the concept of model distillation, read about it. There arise the question, how can we be sure that the "non-robust" features described in the paper are real? Could it be that the effect the paper measures is a model distillation? Think about it: In both the experiments the authors used a trained robust network to label images, on which another network was trained normally. Could it be that the robust network was somehow "leaked"? Think about this possibility, and how it could have happen in experiment #1 and experiment #2.```

## Answer

In distillation training, one model is trained to predict the output probabilities of another model that was trained on an earlier, baseline standard to emphasize accuracy.

In the case of the first experiment, robust model is used to create a robust dataset.
We want to replace each sample x in the original dataset by a sample x_r. x and x_r must be very close in the latent space of a robust model.
We use only the representation layer of the robust model, not the output probabilities and then it is not a distillation training. Furthermore, the new sample x_r does not lie in the latent sapce, but in the input space

Experiment #2 does not use a robust model, but creates advresarial example to show that good classification can be achieved only based on non-robust features

```All in all, we will examine two conjectures:```
1. ```The paper is great, non-robust features are the real thing and different networks use the same features.```
2. ```The paper is wrong, it's only a fancy way of network distilling.```

```Let's first explore the concept of distilling networks. Can we do it in any case? Can you distill a network using only, for example, white noise? Can you do it using only the predictions? Or do you maybe need to use the logits of the network? Answer this question. MNIST might help you with that.```

In [1]:
import numpy as np
from keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Layer

In [2]:
class DivideLayer(Layer):
    def __init__(self, temperature=1, **kwargs):
        self.temperature = temperature
        super(DivideLayer, self).__init__(**kwargs)

    def call(self, inputs, training=None):
        if training:
            return  inputs / self.temperature
        return inputs

In [26]:
def train_model(x_train, y_train, input_shape, num_classes, temperature=1):
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            DivideLayer(temperature),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )
    
    es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=3)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
#     model.summary()
    model.fit(x_train, y_train, batch_size=256, epochs=15, validation_split=0.1, callbacks=[es])
    
    return model

In [29]:
def train_distillation(temperature=1, use_proba=True):
    num_classes = 10
    input_shape = (28, 28, 1)
    
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    teacher = train_model(x_train, y_train, input_shape, num_classes, temperature=temperature)
    
    if use_proba is True:
        train_probas = teacher.predict(x_train)
    
    student = train_model(x_train, train_probas, input_shape, num_classes, temperature=temperature)
    
    return teacher, student

In [30]:
teacher, student = train_distillation(use_proba=True)

Epoch 1/15
422/422 [==============================] - 23s 55ms/step - loss: 0.3593 - accuracy: 0.8892 - val_loss: 0.0810 - val_accuracy: 0.9793
Epoch 2/15
422/422 [==============================] - 23s 56ms/step - loss: 0.1151 - accuracy: 0.9649 - val_loss: 0.0587 - val_accuracy: 0.9832
Epoch 3/15
422/422 [==============================] - 23s 55ms/step - loss: 0.0851 - accuracy: 0.9736 - val_loss: 0.0474 - val_accuracy: 0.9870
Epoch 4/15
422/422 [==============================] - 23s 55ms/step - loss: 0.0726 - accuracy: 0.9769 - val_loss: 0.0465 - val_accuracy: 0.9875
Epoch 5/15
422/422 [==============================] - 23s 55ms/step - loss: 0.0644 - accuracy: 0.9798 - val_loss: 0.0398 - val_accuracy: 0.9890
Epoch 6/15
422/422 [==============================] - 23s 54ms/step - loss: 0.0585 - accuracy: 0.9817 - val_loss: 0.0375 - val_accuracy: 0.9895
Epoch 7/15
422/422 [==============================] - 23s 55ms/step - loss: 0.0531 - accuracy: 0.9836 - val_loss: 0.0339 - val_accuracy:

AttributeError: 'NoneType' object has no attribute 'predict'

```Distilling using only white noise is a though question. Why is it? Think about the concepts of distribution and out-of-distribution in when answering this.```

```Assuming conjecture #2 and regarding experiment #1, the distillation isn't happening on white noise, but not on real images either. Think of an experiment that will help you decide if the phenomenon the authors encountered is indeed just network distillation. Think how to measure your success. Open the hint only if you can't think of a way do to it.```

```You can, for example, take a dataset such as CIFAR10, train a network on 5 classes and try to distill it using the other 5 classes. That way you are using real images, but from different distributions.```

```Let's focus now on conjecture #1. Conduct an experiment that will demonstrate that two different networks indeed use the same features. Do it gradually: Start with two copies of the same architecture and on the same data, and move to different architectures and different subsets of the data (but from the same distribution). Think how to measure your success.```